# Global Rates

In this notebook we compare various yield curves for global markets based on actual market data.

Eurex is a major clearing house facilitating trading in interest rate derivatives. As part of their business Eurex calculates *settlement prices* for interest rate derivatives. In order to ensure transparency for market participants, Eurex publishes the curves used to calculate settlement prices:

https://www.eurex.com/ec-en/clear/eurex-otc-clear/settlement-prices

We can use the settlement prices data file to compare snap shots of market yield curves.

The notebook is structured as follows:

  1. Load Eurex curve data from settlement prices data file and describe content.

  2. Setup QuantLib yield curves from published data.

  3. Calculate par swap rates for various market instruments.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import QuantLib as ql

## Eurex Settlement Price Data

Eurex settlement price data is published as CSV file and can be downloaded from above web site. We assume it is saved in the *data/* sub-folder.

The file name needs to be adjusted to the actual file name from the download.

In [ ]:
eurex_data_file_name = '../data/settlement-prices_20220523.csv'

We use Pandas to read and analyse the data file.

In [ ]:
data = pd.read_csv(eurex_data_file_name)
data.columns

The data represent end-of-day market data for a given valuation date.

In [ ]:
valuation_date_time = data['Value DateTime'].iloc[0]
valuation_date_time

The curve valuation data also represents our valuation date for pricing.

In [ ]:
today = ql.DateParser.parseISO(valuation_date_time[:10])
ql.Settings.instance().evaluationDate = today
today

We can check which curves are included in the data set.

In [ ]:
curve_ids = data['Curve ID'].drop_duplicates()
curve_ids

For each curve we have data as zero coupon bond price $P(0,T)$ and continuously compounded zero rate $z(0,T)$.

In [ ]:
data['Value Type'].drop_duplicates()

We can also check e.g. the maximum maturity date and time.

In [ ]:
display(data['Maturity Date'].max())
display(data['Maturity Offset'].max())
display(data['Maturity Offset'].max()/365) # in years


## QuantLib Yield Curves

We can use the data and construct a QuantLib curve.

In [ ]:
def zero_curve_from_data(curve_id):
    curve_data = data[(data['Curve ID']==curve_id) & (data['Value Type']=='Z') ]
    dates = [ ql.DateParser.parseISO(d[:10]) for d in curve_data['Maturity Date'] ]
    zeros = [ z for z in curve_data['Value'] ]
    curve = ql.ZeroCurve(dates, zeros, ql.Actual365Fixed())
    return ql.YieldTermStructureHandle(curve)

In [ ]:
curve = zero_curve_from_data('EUR.ESTR.1D')
curve

In order to get some intuition of global yield curves we plot and compare various curves.

In [ ]:
def plot_curves(curve_ids, xlim=None, ylim=None):
    plt.figure(figsize=(8,5))
    times = np.linspace(0.0, 30.0, 301)
    for id in curve_ids:
        c = zero_curve_from_data(id)
        z = np.array([ c.zeroRate(T,ql.Continuous,ql.Annual,True).rate() for T in times ])
        plt.plot(times, z*100, label=id)
    plt.legend()
    plt.xlabel('maturity time (in years)')
    plt.ylabel('zero rate (in %)')
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.show()

### EUR Curves

In [ ]:
plot_curves(
    [
    'EUR.ESTR.1D',
    'EUR.EURIBOR.3M',
    'EUR.EURIBOR.6M',
    'EUR.USD_COLL.1D',
    ],
    ylim=(-0.6, 2.0)
)

We find that (as of May 2022) short term EUR rates are still negative. But rates are expected to rise in near future. This is reflected in the steep slope for smaller maturities.

### USD Curves

In [ ]:
plot_curves(
    [
    'USD.SOFR.1D',
    'USD.LIBOR.3M',
    'USD.LIBOR.6M',
    ],
)

USD rates are considerably higher then EUR rates. And we also see a steep increase in the curve for short maturities. This indicates expectations of rate rises in the near future. 

### Emerging Markets Curves

We plot yield curves from some Emerging Market currencies.

| Code | Currency
|------|----------------
| BRL  | Brazilian Real
| CLP  | Chilean Peso
| IDR  | Indonesian Rupiah
| INR  | Indian Rupee


In [ ]:
plot_curves(
    [
    'BRL.ANY.0D',
    'CLP.ANY.0D',
    'IDR.ANY.0D',
    'INR.ANY.0D',
    ],
    xlim=(0,10)
)

We find that Emerging Markets (EM) rates are considerably higher than EUR and USD rates. This is a typical picture because EM countries often also have higher rates of inflation. And the higher nominal interest rates aim at limiting inflation in that countries. Moreover, default risk in EM countries is probably higher compared to US and EUR countries.

## Implied Market Rates

The EUREX data is given as yield curve model parameters (i.e. zero rates and discount factors). We are also interested in corresponding par market rates or par quotes. These par rates represent the data that is usually quoted in the market by market makers.

In order to calculate par rates we need to set up the corresponding market instruments and provide all the details to price these instruments.

We show this exercise for EUR instruments, in particular, EURSTR overnight swaps, 3m/6m Euribor FRAs/swaps and 3m versus 6m tenor basis swaps.

As first step we need indices for our floating rate coupons.

In [ ]:
eur_str    = ql.Eonia(zero_curve_from_data('EUR.ESTR.1D'))  # we can use Eonia as a proxy index; conventions are the same as EURSTR
euribor_3m = ql.Euribor3M(zero_curve_from_data('EUR.EURIBOR.3M'))
euribor_6m = ql.Euribor6M(zero_curve_from_data('EUR.EURIBOR.6M'))

With the indices we can construct the market instruments. Fixed rate and spreads of the instruments are set to zero. These are the quantities which we aim to derive such that the market value of the instrument is zero.

### EURSTR Swaps

In [ ]:
terms = [ '1d', '3m', '6m', '9m', '1y', '18m', '2y', '3y', '4y', '5y', '7y', '10y', '15y', '20y', '25y', '30y' ]
eur_str_swaps = [
    {
        'Term'       : term,
        'Instrument' : ql.MakeOIS(
            ql.Period(term),
            eur_str,
            0.0,
            discountingTermStructure = zero_curve_from_data('EUR.ESTR.1D'),
            ),
    }
    for term in terms
]

### Euribor Swaps

In [ ]:
terms = [ '1y', '18m', '2y', '3y', '4y', '5y', '7y', '10y', '15y', '20y', '25y', '30y' ]

euribor_3m_swaps = [
    {
        'Term'       : term,
        'Instrument' : ql.MakeVanillaSwap(
            ql.Period(term),
            euribor_3m,
            0.0,
            ql.Period('0d'),
            discountingTermStructure = zero_curve_from_data('EUR.ESTR.1D'),
            )
    }
    for term in terms
]

euribor_6m_swaps = [
    {
        'Term'       : term,
        'Instrument' : ql.MakeVanillaSwap(
            ql.Period(term),
            euribor_6m,
            0.0,
            ql.Period('0d'),
            discountingTermStructure = zero_curve_from_data('EUR.ESTR.1D'),
            )
    }
    for term in terms
]

### Euribor FRA's

Forward Rate Agreement (FRA) instrument construction is less convenient. We define a wrapper function that wraps the date calculations. 

In [ ]:
def ql_MakeFra(term, index):
    cal = ql.TARGET()  # we set up Euribor instruments
    spot          = cal.advance(today, ql.Period(index.fixingDays(), ql.Days), ql.Following)
    value_date    = cal.advance(spot, ql.Period(term[:2]), ql.Following)
    maturity_date = cal.advance(value_date, ql.Period(term[3:]), ql.Following)
    return ql.ForwardRateAgreement(index, value_date, maturity_date, ql.Position.Long, 0.0, 1.0, )


terms = [ '0d-3m', '3m-6m', '6m-9m' ]
euribor_3m_fras = [
    {
        'Term'       : term,
        'Instrument' : ql_MakeFra(term, euribor_3m)
    }
    for term in terms
]

terms = [ '0d-6m', '3m-9m' ]
euribor_6m_fras = [
    {
        'Term'       : term,
        'Instrument' : ql_MakeFra(term, euribor_6m)
    }
    for term in terms
]

### Euribor Basis Swaps

Basis swap construction also needs some properties that need to be specified.

In [ ]:
def ql_MakeBasisSwap(term, short_index, long_index):
    cal = ql.TARGET()  # we set up Euribor instruments
    spot          = cal.advance(today, ql.Period(max(short_index.fixingDays(), long_index.fixingDays()), ql.Days), ql.Following)
    maturity_date = cal.advance(spot, ql.Period(term), ql.Following)
    #
    short_schedule = ql.MakeSchedule(
        effectiveDate = spot,
        terminationDate = maturity_date,
        tenor = short_index.tenor(),
        calendar = short_index.fixingCalendar(),
        convention = short_index.businessDayConvention(),
        rule = ql.DateGeneration.Backward,
    )
    short_leg = ql.IborLeg([1.0], short_schedule, short_index)
    #
    long_schedule = ql.MakeSchedule(
        effectiveDate = spot,
        terminationDate = maturity_date,
        tenor = long_index.tenor(),
        calendar = long_index.fixingCalendar(),
        convention = long_index.businessDayConvention(),
        rule = ql.DateGeneration.Backward,
    )
    long_leg = ql.IborLeg([1.0], long_schedule, long_index)
    #
    swap = ql.Swap(short_leg, long_leg)
    engine = ql.DiscountingSwapEngine(zero_curve_from_data('EUR.ESTR.1D'))
    swap.setPricingEngine(engine)
    return swap

terms = [ '1y', '18m', '2y', '3y', '4y', '5y', '7y', '10y', '15y', '20y', '25y', '30y' ]
euribor_3m_6m_swaps = [
    {
        'Term'       : term,
        'Instrument' : ql_MakeBasisSwap(term, euribor_3m, euribor_6m)
    }
    for term in terms
]

### Par Rate Calculation

From the instruments we can now calculate the par rates.

In [ ]:
for swaps in [ eur_str_swaps, euribor_3m_swaps, euribor_6m_swaps ]:
    for i in swaps:
        i['Quote'] = i['Instrument'].fairRate()

for fras in [ euribor_3m_fras, euribor_6m_fras ]:
    for i in fras:
        i['Quote'] = i['Instrument'].forwardRate().rate()

for i in euribor_3m_6m_swaps:
    swap = i['Instrument']
    bp = 1e-4
    i['Quote'] = (swap.legNPV(0) + swap.legNPV(1)) / (-swap.legBPS(0)) * bp

We can check the par rates and compare the par rates to our input zero rates.

In [ ]:
eur_str_swaps = pd.DataFrame(eur_str_swaps).drop(['Instrument'], axis=1)
eur_str_swaps

In [ ]:
euribor_3m_fras = pd.DataFrame(euribor_3m_fras).drop(['Instrument'], axis=1)
euribor_3m_fras

In [ ]:
euribor_3m_swaps = pd.DataFrame(euribor_3m_swaps).drop(['Instrument'], axis=1)
euribor_3m_swaps

In [ ]:
euribor_6m_fras = pd.DataFrame(euribor_6m_fras).drop(['Instrument'], axis=1)
euribor_6m_fras

In [ ]:
euribor_6m_swaps = pd.DataFrame(euribor_6m_swaps).drop(['Instrument'], axis=1)
euribor_6m_swaps

In [ ]:
euribor_3m_6m_swaps = pd.DataFrame(euribor_3m_6m_swaps).drop(['Instrument'], axis=1)
euribor_3m_6m_swaps

Finally, we save the market quotes.

These market quotes will be used to illustrate yield curve calibration. This is essentially the reverse procedure of our par rate calculation example.

In [ ]:
eur_str_swaps.to_csv('../data/eur_str_swaps.csv', index=0, float_format='%.6f')
euribor_3m_fras.to_csv('../data/euribor_3m_fras.csv', index=0, float_format='%.6f')
euribor_3m_swaps.to_csv('../data/euribor_3m_swaps.csv', index=0, float_format='%.6f')
euribor_6m_fras.to_csv('../data/euribor_6m_fras.csv', index=0, float_format='%.6f')
euribor_6m_swaps.to_csv('../data/euribor_6m_swaps.csv', index=0, float_format='%.6f')
euribor_3m_6m_swaps.to_csv('../data/euribor_3m_6m_swaps.csv', index=0, float_format='%.6f')